In [126]:
import pandas as pd
import numpy as np 
import pickle

In [127]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [128]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [142]:
football = pd.read_csv('data/england-transformed.csv')
football = football[football.season >= 2010]
football.head()

,season,week_day,year,month,day,home,visitor,home_current_points,visitor_current_points,home_last_table_position,visitor_last_table_position,home_penultimate_table_position,visitor_penultimate_table_position,result
12342,2010,1,2010,8,14,2,49,3,0,6,17,6,8,2
12343,2010,1,2010,8,14,5,17,3,0,10,8,15,5,2
12344,2010,1,2010,8,14,7,18,1,1,14,12,13,7,1
12345,2010,1,2010,8,14,13,48,3,0,1,21,3,20,2
12346,2010,1,2010,8,14,43,4,1,1,13,9,16,21,1


In [130]:
X = football.iloc[:, :-1].to_numpy()
y = football.iloc[:, -1].to_numpy()

In [131]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [132]:
tcsv = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tcsv.split(X):
    #print(f'Train idx: {train_index}, Test idx: {test_index}')
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = GaussianNB().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, labels=[1,2,3]))

              precision    recall  f1-score   support

           1       0.36      0.12      0.18       171
           2       0.56      0.58      0.57       322
           3       0.38      0.58      0.46       197

    accuracy                           0.47       690
   macro avg       0.44      0.43      0.40       690
weighted avg       0.46      0.47      0.44       690

              precision    recall  f1-score   support

           1       0.22      0.04      0.07       187
           2       0.53      0.63      0.58       319
           3       0.38      0.57      0.45       184

    accuracy                           0.46       690
   macro avg       0.38      0.41      0.37       690
weighted avg       0.41      0.46      0.41       690

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       168
           2       0.54      0.76      0.63       342
           3       0.34      0.41      0.37       180

    accuracy        

In [139]:
tcsv = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tcsv.split(X):
    #print(f'Train idx: {train_index}, Test idx: {test_index}')
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = KNeighborsClassifier(n_neighbors=150).fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, labels=[1,2,3]))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       171
           2       0.47      0.98      0.64       322
           3       0.47      0.04      0.07       197

    accuracy                           0.47       690
   macro avg       0.31      0.34      0.24       690
weighted avg       0.35      0.47      0.32       690

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       187
           2       0.49      0.93      0.64       319
           3       0.45      0.20      0.28       184

    accuracy                           0.48       690
   macro avg       0.31      0.38      0.31       690
weighted avg       0.35      0.48      0.37       690

              precision    recall  f1-score   support

           1       0.25      0.01      0.01       168
           2       0.52      0.92      0.66       342
           3       0.40      0.17      0.24       180

    accuracy        

In [134]:
tcsv = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tcsv.split(X):
    #print(f'Train idx: {train_index}, Test idx: {test_index}')
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = LogisticRegression(multi_class='multinomial').fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, labels=[1,2,3]))

              precision    recall  f1-score   support

           1       0.31      0.06      0.10       171
           2       0.58      0.67      0.62       322
           3       0.44      0.63      0.52       197

    accuracy                           0.51       690
   macro avg       0.44      0.45      0.41       690
weighted avg       0.47      0.51      0.46       690

              precision    recall  f1-score   support

           1       0.17      0.01      0.01       187
           2       0.52      0.74      0.61       319
           3       0.39      0.48      0.43       184

    accuracy                           0.47       690
   macro avg       0.36      0.41      0.35       690
weighted avg       0.39      0.47      0.40       690

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       168
           2       0.52      0.82      0.64       342
           3       0.36      0.31      0.33       180

    accuracy        

In [144]:
model = KNeighborsClassifier(n_neighbors=150).fit(X, y)
pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)